In [70]:
import numpy as np
import pandas as pd

In [71]:
# Explore dataset
data = pd.read_csv('../dataset/winequalityN.csv')
data.head()


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [72]:
data.shape

(6497, 13)

In [73]:
data.columns

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'],
      dtype='object')

In [74]:
data['type'].unique()


array(['white', 'red'], dtype=object)

In [75]:
data.describe().round(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6487.000,6489.000,6494.000,6495.000,6495.000,6497.000,6497.000,6497.000,6488.000,6493.000,6497.000,6497.000
mean,7.217,0.340,0.319,5.444,0.056,30.525,115.745,0.995,3.218,0.531,10.492,5.818
std,1.297,0.165,0.145,4.758,0.035,17.749,56.522,0.003,0.161,0.149,1.193,0.873
min,3.800,0.080,0.000,0.600,0.009,1.000,6.000,0.987,2.720,0.220,8.000,3.000
25%,6.400,0.230,0.250,1.800,0.038,17.000,77.000,0.992,3.110,0.430,9.500,5.000
50%,7.000,0.290,0.310,3.000,0.047,29.000,118.000,0.995,3.210,0.510,10.300,6.000
75%,7.700,0.400,0.390,8.100,0.065,41.000,156.000,0.997,3.320,0.600,11.300,6.000
max,15.900,1.580,1.660,65.800,0.611,289.000,440.000,1.039,4.010,2.000,14.900,9.000


In [76]:
data.isnull().sum(), data.isnull().mean()

(type                     0
 fixed acidity           10
 volatile acidity         8
 citric acid              3
 residual sugar           2
 chlorides                2
 free sulfur dioxide      0
 total sulfur dioxide     0
 density                  0
 pH                       9
 sulphates                4
 alcohol                  0
 quality                  0
 dtype: int64,
 type                    0.000000
 fixed acidity           0.001539
 volatile acidity        0.001231
 citric acid             0.000462
 residual sugar          0.000308
 chlorides               0.000308
 free sulfur dioxide     0.000000
 total sulfur dioxide    0.000000
 density                 0.000000
 pH                      0.001385
 sulphates               0.000616
 alcohol                 0.000000
 quality                 0.000000
 dtype: float64)

After checking the data we see a low percentage of missing values, with a maximum of 0.15% in one column. As it's a large dataset and to make it simply, i'm going to drop the rows with missing values and i should still be able to get a good result.

In [77]:
df = data.dropna().copy()

Now we need to know what columns are really usefull for our prediction

In [78]:
df.groupby('type').mean()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,,,,,,,,,,,,
red,8.326365,0.527659,0.271551,2.538512,0.087497,15.841808,46.463905,0.996752,3.310590,0.657866,10.419617,5.636535
white,6.855123,0.278071,0.334199,6.394343,0.045771,35.317146,138.340144,0.994026,3.188154,0.489700,10.516772,5.878029


In [79]:
# Encode type: red=0, white=1
df.loc[:,'type_encoded'] = df['type'].map({'red': 0, 'white': 1})
# Correlation with type
df_corr= df.select_dtypes(include= 'number')
corr_with_type = df_corr.corr()['type_encoded'].sort_values(ascending=False)
print(corr_with_type)

type_encoded            1.000000
total sulfur dioxide    0.700521
free sulfur dioxide     0.472653
residual sugar          0.349358
citric acid             0.185892
quality                 0.119185
alcohol                 0.035095
pH                     -0.328474
density                -0.391437
sulphates              -0.486715
fixed acidity          -0.488552
chlorides              -0.512705
volatile acidity       -0.653374
Name: type_encoded, dtype: float64


We can see that the most useful features from wine to predict it's type are: total sulfur dioxide, free sulfur dioxide, sulphates, fixed acidity, chlorides and volatile acidity.

Now to end we just need to check that they dont have a high correlation between them, so we don't use irrelevant data

In [80]:
df = df[['total sulfur dioxide', 'free sulfur dioxide', 'sulphates', 'fixed acidity', 'chlorides', 'volatile acidity']]
df.corr()

,total sulfur dioxide,free sulfur dioxide,sulphates,fixed acidity,chlorides,volatile acidity
total sulfur dioxide,1.000000,0.721476,-0.275878,-0.330543,-0.279602,-0.414729
free sulfur dioxide,0.721476,1.000000,-0.188947,-0.283485,-0.195428,-0.353402
sulphates,-0.275878,-0.188947,1.000000,0.301263,0.396240,0.225656
fixed acidity,-0.330543,-0.283485,0.301263,1.000000,0.299104,0.221066
chlorides,-0.279602,-0.195428,0.396240,0.299104,1.000000,0.377995
volatile acidity,-0.414729,-0.353402,0.225656,0.221066,0.377995,1.000000


As total sulfur dioxide and free sulfur dioxide have a high correlation, we are only going to use total sulfure dioxide as it had the higher correlation with the type

In [81]:
df.drop(['free sulfur dioxide'], axis= 'columns', inplace= True)
df.head()

,total sulfur dioxide,sulphates,fixed acidity,chlorides,volatile acidity
0,170.0,0.45,7.0,0.045,0.27
1,132.0,0.49,6.3,0.049,0.30
2,97.0,0.44,8.1,0.050,0.28
3,186.0,0.40,7.2,0.058,0.23
4,186.0,0.40,7.2,0.058,0.23


In [82]:
df.to_csv('clean_wine.csv', index=False)